<a href="https://colab.research.google.com/github/yoonandmoon/GS_team/blob/cleansing_yschoi/231026_GS_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC_4%EC%9B%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#4월_텍스트 전처리

## 토큰화

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/news_data04.csv')

df2 = df[['title','date','content']]
df2['content'] = df['content'].str.replace('\n',' ').replace('\t',' ').replace('\r', ' ')

<ipython-input-9-c0f95ef3e7a4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['content'] = df['content'].str.replace('\n',' ').replace('\t',' ').replace('\r', ' ')


In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.3 MB/s eta 0:00:00


In [ ]:
pip install mecab-ko-msvc mecab-ko-dic-msvc

In [ ]:
import MeCab
import pandas as pd
import re

# Mecab 인스턴스 생성
m = MeCab.Tagger()

# 'title' 열의 결과를 'titl' 열에 저장
df2['titl'] = df2['title'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', x))

# 'content' 열의 결과를 'cont' 열에 저장
df2['cont'] = df2['content'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', x))

def extract_words(text):
    # 텍스트를 Mecab을 사용하여 토큰화합니다.
    tokens = m.parse(text).split('\n')

    # 선택된 단어를 저장할 리스트
    selected_words = []

    for token in tokens:
        if len(token) >= 2:
            # 각 토큰을 탭(\t)으로 분리하여 형태소와 품사 정보를 얻습니다.
            parts = token.split('\t')
            if len(parts) == 2:
                word, pos = parts
                pos = pos.split(',')[0]  # 품사 정보에서 첫 번째 부분만 사용

                # 선택 조건 (명사, 동사, 형용사)을 확인하고 선택된 단어 리스트에 추가
                if len(word) >= 2 and pos in ['NNG', 'VV', 'VA']:
                    selected_words.append(word)

    # 선택된 단어를 공백으로 구분된 문자열로 반환합니다.
    return ' '.join(selected_words)

# 'title' 및 'content' 열에 대해 처리
for i, row in df2.iterrows():
    df2.at[i, 'titl'] = extract_words(row['title'])
    df2.at[i, 'cont'] = extract_words(row['content'])

df3 = df2[['date','titl','cont']]




<ipython-input-11-cfa71b1e9142>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['titl'] = df2['title'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', x))


In [15]:
from collections import Counter

words_list = df3['cont'].str.split()

all_words = [word for words in words_list for word in words]

word_counts = Counter(all_words)

most_common_words = word_counts.most_common(100)
print("가장 많이 나온 상위 50개 단어:")
for word, count in most_common_words:
    print(f"{word}: {count}")


stop_word = ['기자','지난해','올해','따르','기자']

def preprocess(text):
  text = text.split()
  text = [i for i in text if i not in stop_word]
  return text

def preprocess(text):
  text = text.split()
  text = [i for i in text if i not in stop_word]
  return text

def make_tokens(df):
  df['tokens'] = ' '
  for i, row in df.iterrows():
    if i%100==0:
      print(i,'/',len(df))
    token = preprocess(df['cont'][i])
    df['tokens'][i] = ' '.join(token)
  return df

df4 = make_tokens(df3)



가장 많이 나온 상위 50개 단어:
건설: 2544
분양: 1923
가구: 1655
주택: 1590
사업: 1349
아파트: 1051
기술: 956
개발: 914
시장: 902
자이: 869
단지: 728
청약: 725
투자: 720
예정: 698
전용: 656
지난해: 633
부동산: 592
건설사: 590
공급: 585
규모: 580
대표: 564
기업: 548
분기: 542
제공: 487
일반: 480
도시: 467
서비스: 452
대비: 438
광명: 414
국내: 414
벤처: 412
가능: 410
그룹: 409
지역: 408
면적: 395
올해: 392
부문: 392
브랜드: 387
이상: 384
연구: 384
스타트업: 381
영업: 380
기자: 379
시설: 379
조합: 374
진행: 373
현장: 373
순위: 365
계획: 364
따르: 360
이익: 359
모듈러: 358
생산: 358
경쟁: 351
증가: 349
구성: 347
구역: 345
기록: 344
관계자: 340
최근: 339
공사: 335
주행: 335
설계: 334
경기: 331
단독: 331
모듈: 327
환경: 319
계약: 317
조성: 317
자율: 316
매출: 316
유치: 314
최고: 311
평균: 310
지상: 308
산업: 306
물량: 305
적용: 304
이번: 301
기준: 297
차량: 290
타입: 287
지하: 284
업계: 282
공원: 282
전년: 281
상황: 279
세대: 276
평가: 273
마련: 271
경우: 271
주요: 271
수준: 270
기반: 269
기간: 267
가운데: 266
수도: 266
센터: 263
가격: 257
상무: 256
0 / 608
100 / 608
200 / 608


<ipython-input-15-bb5daff8a141>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = ' '
<ipython-input-15-bb5daff8a141>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'][i] = ' '.join(token)


300 / 608
400 / 608
500 / 608
600 / 608


In [ ]:
# 이후 불용어 확인 후 반복 이때는 불용어 처리시 이 코드 사용
#from collections import Counter

#words_list = df4['tokens'].str.split()

#all_words = [word for words in words_list for word in words]

#word_counts = Counter(all_words)

#most_common_words = word_counts.most_common(50)
#print("가장 많이 나온 상위 50개 단어:")
#for word, count in most_common_words:
    #print(f"{word}: {count}")

#날짜만 추출
df4['date'] = df4['date'].str.split(' ').str[0]
print(df4)

#csv파일 만들기
df4.to_csv('4월_전처리.csv', index=False)

           date                            titl  \
0    2023-04-02                 집값 반등 투자 폭탄 우주인   
1    2023-04-02                  규제 완화 분양 자이 계약   
2    2023-04-02               규제 완화 분양 자이 예비 당첨   
3    2023-04-02                     자이 시아 가구 분양   
4    2023-04-02               전국 가구 청약 접수 청약 열기   
..          ...                             ...   
603  2023-04-30            차세대 모빌리티 통사 자율 주행 영토   
604  2023-04-30                 재건축 상징 마아 파트 뚫리   
605  2023-04-30  원자력 수출 용신 연구 건설 착수 동위 원소 국내 자급   
606  2023-04-30  자율 주행 도로 곳곳 청소 도시 환경 관리 서비스 연구   
607  2023-04-30                     역세 자이 타워 공급   

                                                  cont  \
0    미래 증권 수석 연구원 인터뷰 최근 집값 일시 현상 매수 전셋값 거래량 추세 회복 ...   
1    일반 공급 부적격자 물량 계약 완료 영등 포자 디그 투시도 사진 건설 보름 기자 정...   
2    정부 부동산 대책 이후 처음 입주자 모집 공고 눈길 건설 영등 포자 디그 양평 구역...   
3    분양 캘린더 부동산 제공 부동산 전국 단지 가구 일반 분양 가구 분양 시작 주요 분...   
4    아파트 단지 모습 사진 기자 본격 분양 성수기 돌입 가운데 분양 시장 청약 열기 이...   
..                                     

<ipython-input-14-d9f8161f1525>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['date'] = df4['date'].str.split(' ').str[0]
